In [3]:
#!pip install snowflake-snowpark-python[pandas]

In [46]:
import sys
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window

import getpass
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
print("ORGNAME-ACCOUNTNAME (separated by minus sign):")
accountname = getpass.getpass() 
print("SNOWFLAKE-USERNAME:")
username = getpass.getpass()  
print("SNOWFLAKE-PASSWORD:")
password = getpass.getpass() 

ORGNAME-ACCOUNTNAME (separated by minus sign):


 ········


SNOWFLAKE-USERNAME:


 ········


SNOWFLAKE-PASSWORD:


 ········


In [14]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "HOL_DB",
    "schema": "PUBLIC",
    "warehouse": "HOL_WH"
}

session = Session.builder.configs(connection_parameters).create()

## Explore and Prepare Data

![Image](https://www.linkpicture.com/q/descarga_23.png)

In [15]:
# Creating a Snowpark DataFrame
application_record_sdf = session.table('APPLICATION_RECORD')
credit_record_sdf = session.table('CREDIT_RECORD')

In [16]:
# How many records do we have?
application_record_sdf.count()

438557

### Explore and Prepare Data

In [17]:
# Creating a Snowpark DataFrame
application_record_sdf = session.table('APPLICATION_RECORD')
credit_record_sdf = session.table('CREDIT_RECORD')

In [18]:
# How many records do we have?
application_record_sdf.count()

438557

In [19]:
# Check for duplicate records
duplicates_sdf = application_record_sdf.group_by('ID').agg(F.count(('ID'))).filter(F.col('COUNT(ID)') > 1)
print('Number Duplicates:', duplicates_sdf.count())

Number Duplicates: 47


In [20]:
# Lets drop duplicates
application_record_sdf = application_record_sdf.drop_duplicates('ID')
application_record_sdf.count()

438510

In [21]:
# Obtaining simple statistics per column - why are some statistics missing?
# Can you already identify problems in our data?
# Hint: ID variables are useless for ML models (but we need it for joining)
# Hint: Missing values for OCCUPATION_TYPE
# Hint: Max Value for DAYS_EMPLOYED seems wrong
application_record_sdf.describe().to_pandas()

,SUMMARY,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,stddev,5.714962e+05,None,None,None,0.724874,1.100893e+05,None,None,None,None,4185.016222,138770.072835,0.0,0.404523,0.452724,0.310633,None,0.897192
1,mean,6.022035e+06,None,None,None,0.427381,1.875254e+05,None,None,None,None,-15998.022996,60566.188769,1.0,0.206128,0.287770,0.108200,None,2.194463
2,count,4.385100e+05,438510,438510,438510,438510.000000,4.385100e+05,438510,438510,438510,438510,438510.000000,438510.000000,438510.0,438510.000000,438510.000000,438510.000000,304317,438510.000000
3,min,5.008804e+06,F,N,N,0.000000,2.610000e+04,Commercial associate,Academic degree,Civil marriage,Co-op apartment,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,Accountants,1.000000
4,max,7.999952e+06,M,Y,Y,19.000000,6.750000e+06,Working,Secondary / secondary special,Widow,With parents,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,Waiters/barmen staff,20.000000


In [22]:
# We found missing values in OCCUPATION_TYPE - let's impute them
application_record_sdf = application_record_sdf.fillna(value='OTHER', subset=['OCCUPATION_TYPE'])
application_record_sdf.describe().to_pandas()

,SUMMARY,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,count,4.385100e+05,438510,438510,438510,438510.000000,4.385100e+05,438510,438510,438510,438510,438510.000000,438510.000000,438510.0,438510.000000,438510.000000,438510.000000,438510,438510.000000
1,mean,6.022035e+06,None,None,None,0.427381,1.875254e+05,None,None,None,None,-15998.022996,60566.188769,1.0,0.206128,0.287770,0.108200,None,2.194463
2,min,5.008804e+06,F,N,N,0.000000,2.610000e+04,Commercial associate,Academic degree,Civil marriage,Co-op apartment,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,Accountants,1.000000
3,stddev,5.714962e+05,None,None,None,0.724874,1.100893e+05,None,None,None,None,4185.016222,138770.072835,0.0,0.404523,0.452724,0.310633,None,0.897192
4,max,7.999952e+06,M,Y,Y,19.000000,6.750000e+06,Working,Secondary / secondary special,Widow,With parents,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,Waiters/barmen staff,20.000000


In [23]:
# How many distinct values do we have per column?
# Hint: Constant values are probably irrelevant
# Hint: Variables with many different values can be problematic
unique_values = []
for column in application_record_sdf.columns:
    unique_values.append([column, application_record_sdf.select(column).distinct().count()])
pd.DataFrame(unique_values, columns=['COLUMN_NAME','NUM_UNIQUE_VALUES'])

,COLUMN_NAME,NUM_UNIQUE_VALUES
0,ID,438510
1,CODE_GENDER,2
2,FLAG_OWN_CAR,2
3,FLAG_OWN_REALTY,2
4,CNT_CHILDREN,12
5,AMT_INCOME_TOTAL,866
6,NAME_INCOME_TYPE,5
7,NAME_EDUCATION_TYPE,5
8,NAME_FAMILY_STATUS,5
9,NAME_HOUSING_TYPE,6


In [24]:
# We found FLAG_MOBIL to be constant, lets drop it
application_record_sdf = application_record_sdf.drop('FLAG_MOBIL')

In [25]:
credit_record_sdf.describe().to_pandas()

,SUMMARY,ID,MONTHS_BALANCE,STATUS
0,count,1.048575e+06,1.048575e+06,1048575
1,mean,5.068286e+06,-1.913700e+01,None
2,stddev,4.615058e+04,1.402350e+01,None
3,min,5.001711e+06,-6.000000e+01,0
4,max,5.150487e+06,0.000000e+00,X


### Response Variable (Target)

Explanation of Status Variable:
* 0: 1-29 days past due
* 1: 30-59 days past due
* 2: 60-89 days overdue
* 3: 90-119 days overdue
* 4: 120-149 days overdue
* 5: Overdue or bad debts, write-offs for more than 150 days
* C: paid off that month
* X: No loan for the month

We will flag credit customers high-risk (1), if during any month they are late on payments by 60 days or more.
All other customers are low-risk (0).

In [27]:
credit_record_sdf.group_by('STATUS').count().sort('STATUS').to_pandas()

,STATUS,COUNT
0,0,383120
1,1,11090
2,2,868
3,3,320
4,4,223
5,5,1693
6,C,442031
7,X,209230


In [28]:
credit_record_sdf = credit_record_sdf.with_column('TARGET', 
                                                  F.when((F.col('STATUS') == '2') | 
                                                         (F.col('STATUS') == '3') | 
                                                         (F.col('STATUS') == '4') | 
                                                         (F.col('STATUS') == '5'), 'YES'))

In [29]:
cpunt = credit_record_sdf.group_by('ID').agg(F.count('TARGET').as_('TARGET'))
cpunt = cpunt.with_column('TARGET', F.when(F.col('TARGET') > 0, 1).otherwise(0))
cpunt = cpunt.drop('DEP_VALUE')

In [31]:
# Target Variable Distribution
# Hint: Highly imbalanced target variables can be problematic
cpunt.group_by('TARGET').agg(F.count('TARGET')).to_pandas()

,TARGET,COUNT(TARGET)
0,0,45318
1,1,667


In [32]:
# Joining our target variable to our customer records
application_record_sdf = application_record_sdf.join(cpunt, using_columns='ID', join_type='inner')
# Finally we can drop the ID variable as we won't use it for training
application_record_sdf = application_record_sdf.drop('ID')
application_record_sdf.count()

36457

## Feature Engineering
For many ML algorithms we need numeric data, so we'll encode our variables.
Sometimes this requires simple replacement, but sometimes we also need other techniques like buckets.

### Binary Variables

#### Gender

In [33]:
# High Risk Ratio for CODE_GENDER
var_analysis = application_record_sdf.group_by('CODE_GENDER')
var_analysis = var_analysis.agg([F.count('CODE_GENDER').as_('COUNT'), 
                                 F.sum('TARGET').as_('HIGH_RISK')])
var_analysis = var_analysis.with_column('HIGH_RISK_RATIO', F.col('HIGH_RISK') / F.col('COUNT'))
var_analysis.sort('CODE_GENDER').to_pandas()

,CODE_GENDER,COUNT,HIGH_RISK,HIGH_RISK_RATIO
0,F,24430,379,0.015514
1,M,12027,237,0.019706


In [34]:
# Encoding M/F to 0/1
application_record_sdf = application_record_sdf.with_column('CODE_GENDER', 
                                                            F.iff(F.col('CODE_GENDER') == 'F', 0, 1))

### Car

In [35]:
# High Risk Ratio for FLAG_OWN_CAR
var_analysis = application_record_sdf.group_by('FLAG_OWN_CAR')
var_analysis = var_analysis.agg([F.count('FLAG_OWN_CAR').as_('COUNT'), 
                                 F.sum('TARGET').as_('HIGH_RISK')])
var_analysis = var_analysis.with_column('HIGH_RISK_RATIO', F.col('HIGH_RISK') / F.col('COUNT'))
var_analysis.sort('FLAG_OWN_CAR').to_pandas()

,FLAG_OWN_CAR,COUNT,HIGH_RISK,HIGH_RISK_RATIO
0,N,22614,392,0.017334
1,Y,13843,224,0.016181


In [36]:
# Encoding N/Y to 0/1
application_record_sdf = application_record_sdf.with_column('FLAG_OWN_CAR', 
                                                            F.iff(F.col('FLAG_OWN_CAR') == 'N', 0, 1))

### Realty

In [37]:
# High Risk Ratio for FLAG_OWN_CAR
var_analysis = application_record_sdf.group_by('FLAG_OWN_REALTY')
var_analysis = var_analysis.agg([F.count('FLAG_OWN_REALTY').as_('COUNT'), 
                                 F.sum('TARGET').as_('HIGH_RISK')])
var_analysis = var_analysis.with_column('HIGH_RISK_RATIO', F.col('HIGH_RISK') / F.col('COUNT'))
var_analysis.sort('FLAG_OWN_REALTY').to_pandas()

,FLAG_OWN_REALTY,COUNT,HIGH_RISK,HIGH_RISK_RATIO
0,N,11951,251,0.021002
1,Y,24506,365,0.014894


In [38]:
# Encoding N/Y to 0/1
application_record_sdf = application_record_sdf.with_column('FLAG_OWN_REALTY', 
                                                            F.iff(F.col('FLAG_OWN_REALTY') == 'N', 0, 1))

## Continious Variables

### Children

In [40]:
# High Risk Ratio for CNT_CHILDREN
var_analysis = application_record_sdf.group_by('CNT_CHILDREN')
var_analysis = var_analysis.agg([F.count('CNT_CHILDREN').as_('COUNT'), 
                                 F.sum('TARGET').as_('HIGH_RISK')])
var_analysis = var_analysis.with_column('HIGH_RISK_RATIO', F.col('HIGH_RISK') / F.col('COUNT'))
var_analysis.sort('CNT_CHILDREN').to_pandas()

,CNT_CHILDREN,COUNT,HIGH_RISK,HIGH_RISK_RATIO
0,0,25201,431,0.017102
1,1,7492,120,0.016017
2,2,3256,52,0.015971
3,3,419,12,0.028640
4,4,63,1,0.015873
5,5,20,0,0.000000
6,7,2,0,0.000000
7,14,3,0,0.000000
8,19,1,0,0.000000


### One-Hot-Encoding

In [45]:
!git clone https://github.com/Snowflake-Labs/snowpark-python-demos.git

Cloning into 'snowpark-python-demos'...
remote: Enumerating objects: 867, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 867 (delta 29), reused 68 (delta 21), pack-reused 775
Receiving objects: 100% (867/867), 219.99 MiB | 34.82 MiB/s, done.
Resolving deltas: 100% (431/431), done.
Checking out files: 100% (140/140), done.
